In [5]:
import pandas as pd
import json
import numpy as np

In [18]:
df = pd.read_csv('../../../../../Downloads/methods2.tsv', sep='\t', index_col=0)
df

,precision,recall,f1-score,support,split,method,epoch,hash,method_params,task
Unnamed: 0,,,,,,,,,,
COVID-19,0.574970,1.000000,0.730135,36134.000000,0,multigrate_mil,1.0,bcc008b0c3,"{'lr': 1e-05, 'rna_indices_end': 30, 'categori...",hlca_small_embed
Healthy,1.000000,0.002986,0.005954,26791.000000,0,multigrate_mil,1.0,bcc008b0c3,"{'lr': 1e-05, 'rna_indices_end': 30, 'categori...",hlca_small_embed
accuracy,0.575511,0.575511,0.575511,0.575511,0,multigrate_mil,1.0,bcc008b0c3,"{'lr': 1e-05, 'rna_indices_end': 30, 'categori...",hlca_small_embed
macro avg,0.787485,0.501493,0.368045,62925.000000,0,multigrate_mil,1.0,bcc008b0c3,"{'lr': 1e-05, 'rna_indices_end': 30, 'categori...",hlca_small_embed
weighted avg,0.755931,0.575511,0.421807,62925.000000,0,multigrate_mil,1.0,bcc008b0c3,"{'lr': 1e-05, 'rna_indices_end': 30, 'categori...",hlca_small_embed
...,...,...,...,...,...,...,...,...,...,...
0,0.916444,0.887024,0.901494,77627.000000,4,gex_rf,NaN,c496c56996,{'norm': 0},hlca_small_embed
1,0.685663,0.752912,0.717716,25408.000000,4,gex_rf,NaN,c496c56996,{'norm': 0},hlca_small_embed
accuracy,0.853953,0.853953,0.853953,0.853953,4,gex_rf,NaN,c496c56996,{'norm': 0},hlca_small_embed


In [19]:
tasks = ['hlca_small_embed']

In [20]:
best_methods = {}

for task in tasks:
    n_splits = 5
    df_task = df[df['task'] == task]
    best_methods_per_task = {}

    for method in np.unique(df_task['method']):
        df_method = df_task[df_task['method'] == method]
        best_accuracy = -1
        best_hash = None

        # multigrate
        for h in np.unique(df_method['hash']):
            df_tmp = df_method[df_method['hash'] == h]
            # for multigrate need to check if all epochs and query_epochs are present and group by those
            # check only if all splits are present
            if method == 'multigrate_mil':
                for epoch in np.unique(df_tmp['epoch']):
                    df_tmp_epoch = df_tmp[df_tmp['epoch'] == epoch]
                    if len(np.unique(df_tmp_epoch['split'])) != n_splits:
                        continue
                    accuracies = []
                    for i in range(n_splits):
                        accuracies.append(df_tmp_epoch[df_tmp_epoch['split'] == i]['f1-score']['accuracy'])
                        accuracy = np.mean(accuracies)
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_hash = h
                            best_params = df_tmp_epoch['method_params'].iloc[0]
                            best_epoch = epoch
                            best_query_epoch = np.nan
            elif method == 'multigrate':
                for epoch in np.unique(df_tmp['epoch']):
                    df_tmp_epoch = df_tmp[df_tmp['epoch'] == epoch]
                    if len(np.unique(df_tmp_epoch['split'])) != n_splits:
                        continue
                    for query_epoch in np.unique(df_tmp_epoch['query_epoch']):
                        df_tmp_q_epoch = df_tmp_epoch[df_tmp_epoch['epoch'] == query_epoch]
                        if len(np.unique(df_tmp_q_epoch['split'])) != n_splits:
                            continue
                        accuracies = []
                        for i in range(n_splits):
                            accuracies.append(df_tmp_q_epoch[df_tmp_q_epoch['split'] == i]['f1-score']['accuracy'])
                            accuracy = np.mean(accuracies)
                            if accuracy > best_accuracy:
                                best_accuracy = accuracy
                                best_hash = h
                                best_params = df_tmp_q_epoch['method_params'].iloc[0]
                                best_epoch = epoch
                                best_query_epoch = query_epoch

            else:
                if len(np.unique(df_tmp['split'])) != n_splits:
                    continue
                accuracies = []
                for i in range(n_splits):
                    accuracies.append(df_tmp[df_tmp['split'] == i]['f1-score']['accuracy'])
                accuracy = np.mean(accuracies)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_hash = h
                    best_params = df_tmp['method_params'].iloc[0]
                    best_epoch = np.nan
                    best_query_epoch = np.nan

        best_methods_per_task[method] = {
            'hash': best_hash,
            'accuracy': best_accuracy,
            'best_params': best_params,
            'best_epoch': best_epoch,
            'best_query_epoch': best_query_epoch,
        }
    best_df = pd.DataFrame.from_dict(best_methods_per_task).T
    best_df['task'] = task
    best_methods[task] = best_df

best_all = pd.concat(best_methods)

In [21]:
best_all

hash   accuracy   
hlca_small_embed gex_nn          86ea898c60   0.660651  \
                 gex_rf          c496c56996   0.803546   
                 multigrate_mil  bcc008b0c3   0.627043   
                 pb_nn           147a0a3770  75.523594   
                 pb_rf           98a928194f   0.709401   

                                                                       best_params   
hlca_small_embed gex_nn          {'norm': 0, 'epochs': 30, 'lr': 1e-05, 'batch_...  \
                 gex_rf                                                {'norm': 0}   
                 multigrate_mil  {'lr': 1e-05, 'rna_indices_end': 30, 'categori...   
                 pb_nn           {'norm': 0, 'epochs': 200, 'lr': 1e-05, 'batch...   
                 pb_rf                                                 {'norm': 0}   

                                best_epoch best_query_epoch              task  
hlca_small_embed gex_nn                NaN              NaN  hlca_small_embed  
                 gex_rf                NaN              NaN  hlca_small_embed  
                 multigrate_mil        1.0              NaN  hlca_small_embed  
                 pb_nn                 NaN              NaN  hlca_small_embed  
                 pb_rf                 NaN              NaN  hlca_small_embed